In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
text_data = ["I woke up this morning feeling an overwhelming sense of joy. The sun streamed through the window, casting a warm glow that filled me with happiness. As I stepped outside, a gentle breeze brushed against my skin, evoking a sense of calm and contentment. However, as the day progressed, a wave of nostalgia washed over me, reminding me of cherished memories from the past. I found myself smiling at old photographs, feeling a mix of joy and longing.Suddenly, a pang of sadness hit me as I remembered missed opportunities and lost connections. Yet, hope flickered within me, like a small flame refusing to be extinguished. Determination surged through my veins, propelling me forward despite the obstacles. Later, an unexpected surprise lifted my spirits, filling me with excitement and anticipation for what lay ahead. Ultimately, today has been a whirlwind of emotions, each one leaving its mark on my heart."]

In [3]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
labels = input_sequences[:, -1]

In [7]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(labels, num_classes=total_words)

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
5/5 [==============================] - 5s 207ms/step - loss: 4.7096 - accuracy: 0.0268
Epoch 2/100
5/5 [==============================] - 1s 220ms/step - loss: 4.6925 - accuracy: 0.0805
Epoch 3/100
5/5 [==============================] - 1s 212ms/step - loss: 4.6684 - accuracy: 0.0805
Epoch 4/100
5/5 [==============================] - 1s 245ms/step - loss: 4.5804 - accuracy: 0.0537
Epoch 5/100
5/5 [==============================] - 1s 262ms/step - loss: 4.5169 - accuracy: 0.0537
Epoch 6/100
5/5 [==============================] - 1s 259ms/step - loss: 4.4917 - accuracy: 0.0671
Epoch 7/100
5/5 [==============================] - 1s 229ms/step - loss: 4.4576 - accuracy: 0.0537
Epoch 8/100
5/5 [==============================] - 1s 248ms/step - loss: 4.4226 - accuracy: 0.0671
Epoch 9/100
5/5 [==============================] - 1s 220ms/step - loss: 4.3838 - accuracy: 0.0470
Epoch 10/100
5/5 [==============================] - 1s 230ms/step - loss: 4.3207 - accuracy: 0.0671
Epoch 11/

5/5 [==============================] - 1s 279ms/step - loss: 0.6276 - accuracy: 1.0000
Epoch 84/100
5/5 [==============================] - 1s 279ms/step - loss: 0.6121 - accuracy: 1.0000
Epoch 85/100
5/5 [==============================] - 1s 277ms/step - loss: 0.5952 - accuracy: 1.0000
Epoch 86/100
5/5 [==============================] - 2s 292ms/step - loss: 0.5817 - accuracy: 1.0000
Epoch 87/100
5/5 [==============================] - 1s 288ms/step - loss: 0.5654 - accuracy: 1.0000
Epoch 88/100
5/5 [==============================] - 2s 294ms/step - loss: 0.5492 - accuracy: 1.0000
Epoch 89/100
5/5 [==============================] - 1s 290ms/step - loss: 0.5352 - accuracy: 1.0000
Epoch 90/100
5/5 [==============================] - 1s 276ms/step - loss: 0.5261 - accuracy: 1.0000
Epoch 91/100
5/5 [==============================] - 1s 283ms/step - loss: 0.5110 - accuracy: 1.0000
Epoch 92/100
5/5 [==============================] - 1s 288ms/step - loss: 0.4994 - accuracy: 1.0000
Epoch 93/100


In [13]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [14]:
generated_text = generate_text("hello", 10, model, max_sequence_len)
print(generated_text)

hello woke up this morning feeling an overwhelming sense of joy
